In [ ]:
import torch
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels)
        )

    def forward(self, x):
        return x + self.block(x)

# Generator
class Generator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, num_residual_blocks=9):
        super(Generator, self).__init__()
        model = [
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=1, padding=3),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
        ]
        in_features = 64
        for _ in range(2):
            out_features = in_features * 2
            model += [
                nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(in_features)]
        for _ in range(2):
            out_features = in_features // 2
            model += [
                nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features
        model += [nn.Conv2d(64, out_channels, kernel_size=7, stride=1, padding=3), nn.Tanh()]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

# Load the generator model
generator_rain_to_clear = Generator()
generator_rain_to_clear.load_state_dict(torch.load("/Users/matthew/Jupyter/Thesis/DeRain/Epoch-50-Batch-4/generator_rain_to_clear.pth", map_location=torch.device('mps')), strict=False)
generator_rain_to_clear.eval()


# Preprocess the input image
transform = transforms.Compose([
    # transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Load and transform the rainy image
# rainy_image = Image.open("/Users/matthew/Jupyter/Thesis/DeRain/JRDR/rain_data_test_Light/rain/X2/norain-38x2.png")
rainy_image = Image.open("/Users/matthew/Desktop/rainy_image.jpg")
rainy_image_tensor = transform(rainy_image).unsqueeze(0)  # Add batch dimension

# Generate the derained image
with torch.no_grad():
    derained_image_tensor = generator_rain_to_clear(rainy_image_tensor)

# Post-process and display the image
denormalize = transforms.Normalize((-1, -1, -1), (2, 2, 2))  # Reverse normalization
derained_image_tensor = denormalize(derained_image_tensor.squeeze(0))
derained_image = transforms.ToPILImage()(derained_image_tensor)

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title("Rainy Image")
plt.imshow(rainy_image)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Derained Image")
plt.imshow(derained_image)
plt.axis("off")

plt.show()


In [5]:
derained_image.save("derained_image.jpg", format="JPEG")

In [1]:
# ROBOFLOW VIDEO DOWNLOADER

# import os
# from supervision.assets import VideoAssets, download_assets
# if not os.path.exists("data"):
#     os.makedirs("data")
# os.chdir("data")
# download_assets(VideoAssets.VEHICLES)

100%|██████████| 33.7M/33.7M [00:02<00:00, 13.9MB/s]


'vehicles.mp4'

In [2]:
from export_dataset import cache_frames

video_dir = "/Users/matthew/Jupyter/Thesis/I24-3D/video"
data_dir  = "/Users/matthew/Jupyter/Thesis/I24-3D/data"
out_dir   = "/Users/matthew/Jupyter/Thesis/I24-3D/cache"

for scene_id in [1,2,3]:
    ann = Scene(video_dir,data_dir,scene_id = scene_id)
    cache_frames(ann,output_directory = out_dir)

ModuleNotFoundError: No module named 'export_dataset'